# Competición: predicción precios ordenadores (TC: Kaggle)

## Importación librerías

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pprint import *

import re

import warnings
warnings.filterwarnings('ignore')
pd.option_context('mode.use_inf_as_na', True)

from EDA_function_tools import *
from toolbox_DS import *


## Funciones

In [5]:
def columna_discretizada(df,columna, nombre, diccionario):
    df[nombre] = df[columna].map(diccionario)
    mediana = np.median(list(diccionario.values()))
    df[nombre] = df[nombre].fillna(mediana)
    return df

## Carga datos y primer vistazo

In [6]:
df = pd.read_csv('./data/test.csv', index_col=0)
df.head(8)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
id,,,,,,,,,,,,
181,1098,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg
708,330,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8GB,256GB SSD,AMD Radeon RX 540,Windows 10,2.2kg
862,1260,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,500GB HDD,Intel HD Graphics 520,Linux,2.4kg
1064,1137,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.43kg
702,1015,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.34kg
585,1220,Dell,Inspiron 5579,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,512GB SSD,Intel UHD Graphics 620,Windows 10,2kg
195,787,Razer,Blade Pro,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95kg
463,841,Asus,VivoBook Max,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,1TB HDD,Nvidia GeForce 920,Linux,2.1kg


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         391 non-null    int64  
 1   Company           391 non-null    object 
 2   Product           391 non-null    object 
 3   TypeName          391 non-null    object 
 4   Inches            391 non-null    float64
 5   ScreenResolution  391 non-null    object 
 6   Cpu               391 non-null    object 
 7   Ram               391 non-null    object 
 8   Memory            391 non-null    object 
 9   Gpu               391 non-null    object 
 10  OpSys             391 non-null    object 
 11  Weight            391 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 39.7+ KB


## Limpieza y transformación de variables

### Duplicados y nulos

#### Duplicados

In [8]:
df.duplicated().sum()

0

No hay duplicados

#### Nulos

In [9]:
df.isna().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
dtype: int64

No hay nulos

### Tratamiento de variables

In [10]:
# Hago copia del df y elimino las dos columnas comantadas 'laptot_ID' y 'Product'
df_copy = df.copy()
df_copy = df_copy.drop(['laptop_ID','Product'],axis = 1)

#### Variable Ram

In [11]:
# Elimino las letras 'GB' de la varialbe 'Ram'
df_copy['Ram'] = df_copy['Ram'].str.replace('GB','')

#### Variable Weight

In [12]:
# Elimino las letras 'kg' de la variable 'Weight'
df_copy['Weight'] = df_copy['Weight'].str.replace('kg','')

In [13]:
# Reviso los cambios realizados
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
id,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.3
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.2
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.4
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.1


In [14]:
# Veo si se han modificado los tipos de las variables 'Ram' y 'Weight'
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   TypeName          391 non-null    object 
 2   Inches            391 non-null    float64
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Ram               391 non-null    object 
 6   Memory            391 non-null    object 
 7   Gpu               391 non-null    object 
 8   OpSys             391 non-null    object 
 9   Weight            391 non-null    object 
dtypes: float64(1), object(9)
memory usage: 33.6+ KB


In [15]:
# Cambio el tipo de varialbe a 'float' de 'Ram' y 'Weight'
df_copy['Ram'] = df_copy['Ram'].astype('int')
df_copy['Weight'] = df_copy['Weight'].astype('float')

In [16]:
# Compruebo el cambio
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   TypeName          391 non-null    object 
 2   Inches            391 non-null    float64
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Ram               391 non-null    int32  
 6   Memory            391 non-null    object 
 7   Gpu               391 non-null    object 
 8   OpSys             391 non-null    object 
 9   Weight            391 non-null    float64
dtypes: float64(2), int32(1), object(7)
memory usage: 32.1+ KB


#### Variable ScreenResolution

In [17]:
# Obtengo la resolución en valores 
resolution = []
for resolucion in df_copy['ScreenResolution']:
    x = re.findall('\d\d\d\dx\d\d\d?\d',resolucion)
    resolution.append(x)
print(resolution)

[['3840x2160'], ['1366x768'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2560x1440'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2560x1440'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2736x1824'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1366x768'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1366x768'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2160x1440'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['3200x1800'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['2256x1504'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1920x1080'], ['1366x768'], ['1920x

In [18]:
# Obtengo una lista única (dado que tenía una lista de listas)
lista_resolution = []
for n in range(0, len(resolution)):
    valor = resolution[n][0]
    lista_resolution.append(valor)
print(lista_resolution)


['3840x2160', '1366x768', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2560x1440', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2560x1440', '1920x1080', '1920x1080', '1920x1080', '3200x1800', '1920x1080', '1920x1080', '1920x1080', '2736x1824', '1920x1080', '1366x768', '1920x1080', '1366x768', '3200x1800', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1366x768', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '2160x1440', '1920x1080', '1920x1080', '1366x768', '3200x1800', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1920x1080', '1920x1080', '2256x1504', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1366x768', '1920x1080', '1600x900', '1920x1080', '1600x900', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1920x1080', '1366x768', '

In [19]:
# Añado nueva variable con la información
df_copy['Resolution'] = lista_resolution

In [20]:
# Compruebo
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution
id,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080


In [21]:
dicc_resolution = {'1366x768': 0.0,
 '1440x900': 3.0,
 '1600x900': 1.0,
 '1920x1080': 3.0,
 '1920x1200': 0.0,
 '2160x1440': 4.0,
 '2256x1504': 4.0,
 '2304x1440': 4.0,
 '2400x1600': 7.0,
 '2560x1440': 9.0,
 '2560x1600': 7.0,
 '2880x1800': 9.0,
 '3200x1800': 5.0,
 '3840x2160': 9.0}

In [22]:
columna_discretizada(df_copy, 'Resolution','Resolution_disc',dicc_resolution)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc
id,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,1920x1080,3.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,1920x1080,3.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,1366x768,0.0


In [23]:
tipos_pantalla =[]
screen_types = {'Pantalla':['4K Ultra HD', 'Full HD', 'Quad HD+', 'Retina Display','Touchscreen']}

for n in df_copy['ScreenResolution']:
    found = False  
    for p in screen_types['Pantalla']:
        if p in n:
            tipos_pantalla.append(p)
            found = True
            break  
    if not found:
        tipos_pantalla.append('Full HD')

print(tipos_pantalla)


['4K Ultra HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Retina Display', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Quad HD+', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Full HD', 'Touchscreen', 'Full HD', 'Full HD', '

In [24]:
df_copy['Pantalla'] = tipos_pantalla

In [25]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc,Pantalla
id,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0,4K Ultra HD
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0,Full HD
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0,Full HD
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0,Full HD
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0,Full HD
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080,3.0,Full HD
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080,3.0,Full HD
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080,3.0,Full HD


In [26]:
# Al parecer no he hecho esta discretización en el train
# Lo dejo preparado por si acaso
dicc_pantalla = {'Touchscreen': 0,
 'Full HD': 1,
 'Quad HD+': 2,
 'Retina Display': 3,
 '4K Ultra HD': 4}

columna_discretizada(df_copy, 'Pantalla','Pantalla_disc',dicc_pantalla)


,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc,Pantalla,Pantalla_disc
id,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0,4K Ultra HD,4
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0,Full HD,1
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0,Full HD,1
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0,Full HD,1
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0,Full HD,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,1920x1080,3.0,Full HD,1
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,1920x1080,3.0,Full HD,1
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,1366x768,0.0,Full HD,1


#### Variable Cpu

In [27]:
# Función para extraer el fabricante (brand)
def extract_brand(cpu):
    return cpu.split()[0]

# Función para extraer la familia (family)
def extract_family(cpu):
    parts = cpu.split()
    if parts[1] == 'Core':
        return parts[1] + ' ' + parts[2]
    else:
        return parts[1]

# Función para extraer el modelo
def extract_model(cpu):
    parts = cpu.split()
    if parts[1] == 'Core':
        return parts[3]
    else:
        return parts[2]

# Función para extraer la velocidad de reloj (clock speed)
def extract_clock_speed(cpu):
    return float(cpu.split()[-1].replace('GHz', ''))

# Aplicar las funciones para crear nuevas columnas
df_copy['Brand'] = df_copy['Cpu'].apply(extract_brand)
df_copy['Family'] = df_copy['Cpu'].apply(extract_family)
df_copy['Model'] = df_copy['Cpu'].apply(extract_model)
df_copy['Clock Speed (GHz)'] = df_copy['Cpu'].apply(extract_clock_speed)

In [28]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc,Pantalla,Pantalla_disc,Brand,Family,Model,Clock Speed (GHz)
id,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0,4K Ultra HD,4,Intel,Core i7,7500U,2.7
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0,Full HD,1,AMD,A12-Series,9720P,2.7
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0,Full HD,1,Intel,Core i3,6006U,2.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0,Full HD,1,Intel,Core i5,6200U,2.3
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0,Full HD,1,Intel,Core i5,7200U,2.5
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,1920x1080,3.0,Full HD,1,Intel,Core i7,8550U,1.8
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,1920x1080,3.0,Full HD,1,Intel,Core i7,7700HQ,2.8
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,1920x1080,3.0,Full HD,1,Intel,Core i5,7200U,2.5


In [29]:
dicc_family_procesador = {'A6-Series': 0.0,
 'Atom': 0.0,
 'Celeron': 0.0,
 'E-Series': 0.0,
 'Pentium': 0.0,
 'A8-Series': 1.0,
 'A9-Series': 1.0,
 'Core i3': 1.0,
 'A10-Series': 2.0,
 'A12-Series': 2.0,
 'Core i5': 2.0,
 'FX': 2.0,
 'Core M': 3.0,
 'Core i7': 3.0,
 'Ryzen': 3.0,
 'Xeon': 4.0}

In [30]:
columna_discretizada(df_copy,'Family','Family_disc',dicc_family_procesador)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc,Pantalla,Pantalla_disc,Brand,Family,Model,Clock Speed (GHz),Family_disc
id,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0,4K Ultra HD,4,Intel,Core i7,7500U,2.7,3.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0,Full HD,1,AMD,A12-Series,9720P,2.7,2.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0,Full HD,1,Intel,Core i3,6006U,2.0,1.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0,Full HD,1,Intel,Core i5,6200U,2.3,2.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,1920x1080,3.0,Full HD,1,Intel,Core i7,7700HQ,2.8,3.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,1920x1080,3.0,Full HD,1,Intel,Core i7,6700HQ,2.6,3.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,1366x768,0.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0


Esta variable es interesante. Si la dejo como numérica, resulta que el modelo interpretará que a mayor número, más importancia, pero sin embargo, resulta que no es así...

In [31]:
dicc_clock_speed = {2.1: 0.0,
 3.0: 0.0,
 2.2: 0.0,
 2.0: 0.0,
 1.92: 0.0,
 1.5: 0.0,
 3.6: 0.0,
 1.44: 0.0,
 1.1: 0.0,
 1.6: 1.0,
 1.0: 1.0,
 2.4: 1.0,
 0.9: 1.0,
 2.5: 2.0,
 2.3: 3.0,
 1.8: 3.0,
 1.9: 4.0,
 2.7: 4.0,
 1.2: 4.0,
 3.2: 6.0,
 1.3: 6.0,
 2.6: 6.0,
 2.8: 6.0,
 3.1: 8.0,
 2.9: 10.0}

In [32]:
columna_discretizada(df_copy,'Clock Speed (GHz)','Clock_speed_disc',dicc_clock_speed)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Resolution,Resolution_disc,Pantalla,Pantalla_disc,Brand,Family,Model,Clock Speed (GHz),Family_disc,Clock_speed_disc
id,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,3840x2160,9.0,4K Ultra HD,4,Intel,Core i7,7500U,2.7,3.0,4.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,1366x768,0.0,Full HD,1,AMD,A12-Series,9720P,2.7,2.0,4.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,1366x768,0.0,Full HD,1,Intel,Core i3,6006U,2.0,1.0,0.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,1920x1080,3.0,Full HD,1,Intel,Core i5,6200U,2.3,2.0,3.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,1920x1080,3.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,1920x1080,3.0,Full HD,1,Intel,Core i7,7700HQ,2.8,3.0,6.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,1920x1080,3.0,Full HD,1,Intel,Core i7,6700HQ,2.6,3.0,6.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,1366x768,0.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0,2.0


In [33]:
dicc_model_procesador = {'x5-Z8550': 0.0,
 'A10-9620P': 0.0,
 'A6-9220': 0.0,
 '9420': 0.0,
 'Dual': 0.0,
 '7100U': 0.0,
 '7110': 0.0,
 '6110': 0.0,
 '6006U': 0.0,
 '9720P': 0.0,
 'E2-9000': 0.0,
 'E2-6110': 0.0,
 '7410': 0.0,
 'Quad': 0.0,
 'X5-Z8350': 0.0,
 'Z8350': 0.0,
 '9220': 0.0,
 '9000e': 0.0,
 'x5-Z8350': 0.0,
 '9000': 0.0,
 'x5-Z8300': 0.0,
 '9410': 0.0,
 '7130U': 1.0,
 '6Y30': 1.0,
 '9620P': 1.0,
 '9830P': 1.0,
 '7Y30': 1.0,
 'A9-9420': 1.0,
 '6100U': 1.0,
 '9700P': 1.0,
 'M3-6Y30': 1.0,
 '8800P': 2.0,
 'm3-7Y30': 2.0,
 '1.6GHz': 2.0,
 '8250U': 2.0,
 '7300HQ': 2.0,
 '7200U': 2.0,
 '1.8GHz': 3.0,
 '6200U': 3.0,
 '6300HQ': 3.0,
 '8550U': 3.0,
 'M7-6Y75': 3.0,
 '7440HQ': 3.0,
 'm3': 4.0,
 '8650U': 4.0,
 '1.1GHz': 4.0,
 '1.2GHz': 4.0,
 '7560U': 4.0,
 '7500U': 4.0,
 '7300U': 4.0,
 '6300U': 4.0,
 '6500U': 4.0,
 '7Y54': 5.0,
 '7820HQ': 5.0,
 '1.3GHz': 5.0,
 '1700': 5.0,
 '2.3GHz': 5.0,
 '6700HQ': 5.0,
 '7Y57': 5.0,
 '6560U': 6.0,
 '1600': 6.0,
 '7600U': 6.0,
 '7700HQ': 6.0,
 '3.1GHz': 7.0,
 '2.9GHz': 7.0,
 '6Y75': 7.0,
 '6600U': 7.0,
 '7Y75': 7.0,
 '2.2GHz': 8.0,
 '6820HQ': 9.0,
 '2.8GHz': 9.0,
 '7660U': 9.0,
 '6820HK': 10.0,
 'E3-1505M': 10.0,
 '2.7GHz': 10.0,
 '6920HQ': 10.0,
 '7820HK': 10.0,
 'E3-1535M': 12.0}

In [34]:
columna_discretizada(df_copy,'Model','Modelo_procesador_disc',dicc_model_procesador)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Resolution_disc,Pantalla,Pantalla_disc,Brand,Family,Model,Clock Speed (GHz),Family_disc,Clock_speed_disc,Modelo_procesador_disc
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,9.0,4K Ultra HD,4,Intel,Core i7,7500U,2.7,3.0,4.0,4.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,0.0,Full HD,1,AMD,A12-Series,9720P,2.7,2.0,4.0,0.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,0.0,Full HD,1,Intel,Core i3,6006U,2.0,1.0,0.0,0.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,3.0,Full HD,1,Intel,Core i5,6200U,2.3,2.0,3.0,3.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,3.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,...,3.0,Full HD,1,Intel,Core i7,7700HQ,2.8,3.0,6.0,6.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,...,3.0,Full HD,1,Intel,Core i7,6700HQ,2.6,3.0,6.0,5.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,...,0.0,Full HD,1,Intel,Core i5,7200U,2.5,2.0,2.0,2.0


#### Variable Gpu

In [35]:
# Función para extraer el fabricante (brand)
def extract_gpu_brand(gpu):
    return gpu.split()[0]

# Función para extraer el modelo
def extract_gpu_model(gpu):
    return ' '.join(gpu.split()[1:])

# Crear una lista de GPUs integradas y dedicadas para clasificar el tipo
integrated_gpus = ['Intel HD Graphics', 'Intel UHD Graphics', 'Intel Iris Plus Graphics', 'Intel Iris Graphics', 'Intel HD Graphics 5000', 'Intel HD Graphics 6000', 'Intel HD Graphics 5200', 'Intel Iris Pro Graphics', 'Intel Iris Graphics', 'Intel UHD Graphics']
dedicated_gpus = ['Nvidia GeForce', 'Nvidia Quadro', 'AMD Radeon', 'AMD FirePro']

# Función para clasificar el tipo de GPU
def classify_gpu_type(gpu):
    brand_model = ' '.join(gpu.split()[:2])
    if any(integrated in brand_model for integrated in integrated_gpus):
        return 'Integrated'
    elif any(dedicated in brand_model for dedicated in dedicated_gpus):
        return 'Dedicated'
    else:
        return 'Unknown'

# Aplicar las funciones para crear nuevas columnas
df_copy['Gpu Brand'] = df_copy['Gpu'].apply(extract_gpu_brand)
df_copy['Gpu Model'] = df_copy['Gpu'].apply(extract_gpu_model)
df_copy['Gpu Type'] = df_copy['Gpu'].apply(classify_gpu_type)

In [36]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Brand,Family,Model,Clock Speed (GHz),Family_disc,Clock_speed_disc,Modelo_procesador_disc,Gpu Brand,Gpu Model,Gpu Type
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,Intel,Core i7,7500U,2.7,3.0,4.0,4.0,Intel,HD Graphics 620,Unknown
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,AMD,A12-Series,9720P,2.7,2.0,4.0,0.0,AMD,Radeon RX 540,Dedicated
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,Intel,Core i3,6006U,2.0,1.0,0.0,0.0,Intel,HD Graphics 520,Unknown
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,Intel,Core i5,6200U,2.3,2.0,3.0,3.0,Intel,HD Graphics 520,Unknown
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,Intel,Core i5,7200U,2.5,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,...,Intel,Core i7,8550U,1.8,3.0,3.0,3.0,Intel,UHD Graphics 620,Unknown
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,...,Intel,Core i7,7700HQ,2.8,3.0,6.0,6.0,Nvidia,GeForce GTX 1060,Dedicated
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,...,Intel,Core i5,7200U,2.5,2.0,2.0,2.0,Nvidia,GeForce 920,Dedicated


In [37]:
dicc_gpu_brand = {'AMD':0,'Intel':1,'Nvidia':2}

columna_discretizada(df_copy,'Gpu Brand','Gpu_Brand_disc',dicc_gpu_brand)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Family,Model,Clock Speed (GHz),Family_disc,Clock_speed_disc,Modelo_procesador_disc,Gpu Brand,Gpu Model,Gpu Type,Gpu_Brand_disc
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,Core i7,7500U,2.7,3.0,4.0,4.0,Intel,HD Graphics 620,Unknown,1.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,A12-Series,9720P,2.7,2.0,4.0,0.0,AMD,Radeon RX 540,Dedicated,0.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,Core i3,6006U,2.0,1.0,0.0,0.0,Intel,HD Graphics 520,Unknown,1.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,Core i5,6200U,2.3,2.0,3.0,3.0,Intel,HD Graphics 520,Unknown,1.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,Core i5,7200U,2.5,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,...,Core i7,7700HQ,2.8,3.0,6.0,6.0,Nvidia,GeForce GTX 1050M,Dedicated,2.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,...,Core i7,6700HQ,2.6,3.0,6.0,5.0,Nvidia,GeForce GTX 960,Dedicated,2.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,...,Core i5,7200U,2.5,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0


In [38]:
dicc_gpu_model = {'Radeon R5 M420': 0.0,
 'HD Graphics 405': 0.0,
 'HD Graphics 500': 0.0,
 'R4 Graphics': 0.0,
 'Radeon R5 M420X': 0.0,
 'Radeon R5 M430': 0.0,
 'HD Graphics 505': 0.0,
 'HD Graphics 510': 0.0,
 'GeForce GTX 940MX': 0.0,
 'Radeon R7': 0.0,
 'HD Graphics': 0.0,
 'Radeon R7 M440': 0.0,
 'HD Graphics 400': 0.0,
 'Radeon R2': 0.0,
 'Radeon R5': 0.0,
 'Radeon R4 Graphics': 0.0,
 'Radeon R4': 0.0,
 'Radeon R2 Graphics': 0.0,
 'Radeon RX 560': 1.0,
 'Radeon 520': 1.0,
 'GeForce MX130': 1.0,
 'Radeon R5 M330': 1.0,
 'Radeon 530': 1.0,
 'GeForce 920M': 1.0,
 'Radeon R5 520': 1.0,
 'Radeon R5 M315': 1.0,
 'GeForce GTX 940M': 1.0,
 'Radeon R7 Graphics': 1.0,
 'GeForce 920MX': 1.0,
 'HD Graphics 620': 2.0,
 'HD Graphics 6000': 2.0,
 'HD Graphics 520': 2.0,
 'UHD Graphics 620': 2.0,
 'GeForce MX150': 2.0,
 'Radeon R9 M385': 2.0,
 'Radeon R7 M460': 2.0,
 'GeForce 940MX': 2.0,
 'Radeon R7 M445': 2.0,
 'Graphics 620': 2.0,
 'R17M-M1-70': 2.0,
 'GeForce GTX 950M': 2.0,
 'GeForce GTX 1050Ti': 3.0,
 'HD Graphics 5300': 3.0,
 'GeForce GT 940MX': 3.0,
 'Radeon R7 M360': 3.0,
 'GeForce 930MX': 3.0,
 'GeForce GTX 960<U+039C>': 3.0,
 'GeForce GTX 960M': 3.0,
 'Radeon RX 550': 3.0,
 'GeForce GTX 1050': 3.0,
 'Radeon R7 M365X': 4.0,
 'FirePro W4190M': 4.0,
 'HD Graphics 630': 4.0,
 'GeForce 150MX': 4.0,
 'GeForce GTX 1050 Ti': 4.0,
 'GeForce GTX 965M': 4.0,
 'HD Graphics 515': 4.0,
 'GeForce 930M': 5.0,
 'GeForce GTX1050 Ti': 5.0,
 'HD Graphics 615': 5.0,
 'Radeon RX 580': 5.0,
 'Iris Plus Graphics 640': 5.0,
 'GeForce GTX 1060': 6.0,
 'Iris Graphics 540': 6.0,
 'Quadro M620': 7.0,
 'Iris Plus Graphics 650': 7.0,
 'Iris Graphics 550': 7.0,
 'Quadro M500M': 8.0,
 'GeForce GTX 970M': 8.0,
 'Quadro M1200': 8.0,
 'Quadro M2200': 8.0,
 'Iris Pro Graphics': 8.0,
 'Radeon Pro 555': 9.0,
 'Quadro M1000M': 9.0,
 'GeForce 960M': 9.0,
 'Quadro M620M': 10.0,
 'GeForce GTX 980M': 10.0,
 'Quadro 3000M': 10.0,
 'Radeon Pro 455': 10.0,
 'FirePro W6150M': 10.0,
 'GTX 980 SLI': 10.0,
 'GeForce GTX 1070': 10.0,
 'Quadro M2000M': 11.0,
 'GeForce GTX 1070M': 11.0,
 'Quadro M3000M': 11.0,
 'Quadro M520M': 11.0,
 'Quadro M2200M': 12.0,
 'GeForce GTX 1080': 12.0}

In [39]:
columna_discretizada(df_copy,'Gpu Model','GPU_model_disc',dicc_gpu_model)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Model,Clock Speed (GHz),Family_disc,Clock_speed_disc,Modelo_procesador_disc,Gpu Brand,Gpu Model,Gpu Type,Gpu_Brand_disc,GPU_model_disc
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,7500U,2.7,3.0,4.0,4.0,Intel,HD Graphics 620,Unknown,1.0,2.0
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,9720P,2.7,2.0,4.0,0.0,AMD,Radeon RX 540,Dedicated,0.0,3.0
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,6006U,2.0,1.0,0.0,0.0,Intel,HD Graphics 520,Unknown,1.0,2.0
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,6200U,2.3,2.0,3.0,3.0,Intel,HD Graphics 520,Unknown,1.0,2.0
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,7200U,2.5,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,...,7700HQ,2.8,3.0,6.0,6.0,Nvidia,GeForce GTX 1050M,Dedicated,2.0,3.0
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,...,6700HQ,2.6,3.0,6.0,5.0,Nvidia,GeForce GTX 960,Dedicated,2.0,3.0
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,...,7200U,2.5,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0,2.0


#### Variable Memory

In [40]:
# Función para extraer el tipo de almacenamiento (Storage Type)
def extract_storage_type(memory):
    if 'SSD' in memory:
        return 'SSD'
    elif 'HDD' in memory:
        return 'HDD'
    elif 'Hybrid' in memory:
        return 'Hybrid'
    elif 'Flash Storage' in memory:
        return 'Flash Storage'
    else:
        return 'Unknown'

# Función para extraer la capacidad de almacenamiento (Storage Capacity)
def extract_storage_capacity(memory):
    # Buscar el primer número seguido opcionalmente por 'GB' o 'TB'
    match = re.search(r'(\d+)(?:GB|TB)', memory)
    if match:
        # Extraer el número encontrado en el grupo de captura 1
        number = int(match.group(1))
        medida = match.group(0)[-2:]
        # Verificar si hay 'TB' o 'GB' en la cadena y ajustar la capacidad
        if 'TB' in medida:
            number *= 1000
        return number
    return None

# Aplicar las funciones para crear nuevas columnas
df_copy['Storage Type'] = df_copy['Memory'].apply(extract_storage_type)
df_copy['Storage Capacity (GB)'] = df_copy['Memory'].apply(extract_storage_capacity)

In [41]:
df_copy.head(8)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Family_disc,Clock_speed_disc,Modelo_procesador_disc,Gpu Brand,Gpu Model,Gpu Type,Gpu_Brand_disc,GPU_model_disc,Storage Type,Storage Capacity (GB)
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,3.0,4.0,4.0,Intel,HD Graphics 620,Unknown,1.0,2.0,SSD,512
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,2.0,4.0,0.0,AMD,Radeon RX 540,Dedicated,0.0,3.0,SSD,256
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,1.0,0.0,0.0,Intel,HD Graphics 520,Unknown,1.0,2.0,HDD,500
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,2.0,3.0,3.0,Intel,HD Graphics 520,Unknown,1.0,2.0,SSD,256
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,2.0,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0,2.0,SSD,256
585,Dell,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 8550U 1.8GHz,16,512GB SSD,Intel UHD Graphics 620,Windows 10,2.00,...,3.0,3.0,3.0,Intel,UHD Graphics 620,Unknown,1.0,2.0,SSD,512
195,Razer,Gaming,14.0,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,512GB SSD,Nvidia GeForce GTX 1060,Windows 10,1.95,...,3.0,6.0,6.0,Nvidia,GeForce GTX 1060,Dedicated,2.0,6.0,SSD,512
463,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4,1TB HDD,Nvidia GeForce 920,Linux,2.10,...,2.0,2.0,2.0,Nvidia,GeForce 920,Dedicated,2.0,3.0,HDD,1000


In [42]:
dicc_storage_type = {'Flash Storage':0,'HDD':1,'Hybrid':2,'SSD':3}
columna_discretizada(df_copy,'Storage Type','storage_type_disc',dicc_storage_type)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,...,Clock_speed_disc,Modelo_procesador_disc,Gpu Brand,Gpu Model,Gpu Type,Gpu_Brand_disc,GPU_model_disc,Storage Type,Storage Capacity (GB),storage_type_disc
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,1.30,...,4.0,4.0,Intel,HD Graphics 620,Unknown,1.0,2.0,SSD,512,3
708,Acer,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,2.20,...,4.0,0.0,AMD,Radeon RX 540,Dedicated,0.0,3.0,SSD,256,3
862,Acer,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,2.40,...,0.0,0.0,Intel,HD Graphics 520,Unknown,1.0,2.0,HDD,500,1
1064,HP,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,1.43,...,3.0,3.0,Intel,HD Graphics 520,Unknown,1.0,2.0,SSD,256,3
702,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,1.34,...,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0,2.0,SSD,256,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,2.40,...,6.0,6.0,Nvidia,GeForce GTX 1050M,Dedicated,2.0,3.0,SSD,256,3
524,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,3.31,...,6.0,5.0,Nvidia,GeForce GTX 960,Dedicated,2.0,3.0,SSD,512,3
1015,HP,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,1.96,...,2.0,2.0,Intel,HD Graphics 620,Unknown,1.0,2.0,HDD,500,1


Cargo la variable de componentes principales


In [43]:
df_copy['Componentes_principales'] = df_copy[['Ram','Clock_speed_disc','GPU_model_disc','storage_type_disc','Modelo_procesador_disc']].apply(sum,axis=1)
df_copy['Componentes_principales'] = df_copy['Componentes_principales'].apply(np.log1p)

### DataFrame final

In [44]:
df_copy.columns

Index(['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram',
       'Memory', 'Gpu', 'OpSys', 'Weight', 'Resolution', 'Resolution_disc',
       'Pantalla', 'Pantalla_disc', 'Brand', 'Family', 'Model',
       'Clock Speed (GHz)', 'Family_disc', 'Clock_speed_disc',
       'Modelo_procesador_disc', 'Gpu Brand', 'Gpu Model', 'Gpu Type',
       'Gpu_Brand_disc', 'GPU_model_disc', 'Storage Type',
       'Storage Capacity (GB)', 'storage_type_disc',
       'Componentes_principales'],
      dtype='object')

In [45]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Company                  391 non-null    object 
 1   TypeName                 391 non-null    object 
 2   Inches                   391 non-null    float64
 3   ScreenResolution         391 non-null    object 
 4   Cpu                      391 non-null    object 
 5   Ram                      391 non-null    int32  
 6   Memory                   391 non-null    object 
 7   Gpu                      391 non-null    object 
 8   OpSys                    391 non-null    object 
 9   Weight                   391 non-null    float64
 10  Resolution               391 non-null    object 
 11  Resolution_disc          391 non-null    float64
 12  Pantalla                 391 non-null    object 
 13  Pantalla_disc            391 non-null    int64  
 14  Brand                    391

#### Selección de columnas (df_disc_test)

Voy a eliminar las columnas creadas de la mediana de los importes así como las columnas que son sustituidas por las columnas discretizadas así como columnas que han sido tratadas y por lo tanto la información está en otras.

In [46]:
columnas_eliminar_df_disc_test = ['ScreenResolution','Cpu','Memory','Gpu','Resolution','Pantalla','Brand',
'Family','Model','Clock Speed (GHz)','Gpu Brand','Gpu Model',
'Storage Type','Storage Capacity (GB)']

In [47]:
df_disc = df_copy.drop(columns=columnas_eliminar_df_disc_test)
df_disc.head(8)

,Company,TypeName,Inches,Ram,OpSys,Weight,Resolution_disc,Pantalla_disc,Family_disc,Clock_speed_disc,Modelo_procesador_disc,Gpu Type,Gpu_Brand_disc,GPU_model_disc,storage_type_disc,Componentes_principales
id,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,16,Windows 10,1.30,9.0,4,3.0,4.0,4.0,Unknown,1.0,2.0,3,3.401197
708,Acer,Notebook,15.6,8,Windows 10,2.20,0.0,1,2.0,4.0,0.0,Dedicated,0.0,3.0,3,2.944439
862,Acer,Notebook,15.6,4,Linux,2.40,0.0,1,1.0,0.0,0.0,Unknown,1.0,2.0,1,2.079442
1064,HP,Notebook,14.0,8,Windows 7,1.43,3.0,1,2.0,3.0,3.0,Unknown,1.0,2.0,3,2.995732
702,HP,Notebook,13.3,8,Windows 10,1.34,3.0,1,2.0,2.0,2.0,Unknown,1.0,2.0,3,2.890372
585,Dell,2 in 1 Convertible,15.6,16,Windows 10,2.00,3.0,1,3.0,3.0,3.0,Unknown,1.0,2.0,3,3.332205
195,Razer,Gaming,14.0,16,Windows 10,1.95,3.0,1,3.0,6.0,6.0,Dedicated,2.0,6.0,3,3.637586
463,Asus,Notebook,15.6,4,Linux,2.10,3.0,1,2.0,2.0,2.0,Dedicated,2.0,3.0,1,2.564949


#### Seleccion columnas (df_copy)

In [48]:
df_copy.columns

Index(['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram',
       'Memory', 'Gpu', 'OpSys', 'Weight', 'Resolution', 'Resolution_disc',
       'Pantalla', 'Pantalla_disc', 'Brand', 'Family', 'Model',
       'Clock Speed (GHz)', 'Family_disc', 'Clock_speed_disc',
       'Modelo_procesador_disc', 'Gpu Brand', 'Gpu Model', 'Gpu Type',
       'Gpu_Brand_disc', 'GPU_model_disc', 'Storage Type',
       'Storage Capacity (GB)', 'storage_type_disc',
       'Componentes_principales'],
      dtype='object')

In [49]:
columnas_eliminar_df_copy_test = ['ScreenResolution', 'Cpu',
       'Memory', 'Gpu',
       'Resolution_disc', 'Family_disc', 'Clock_speed_disc', 'Modelo_procesador_disc',
       'Gpu_Brand_disc', 'GPU_model_disc',
       'Storage Capacity (GB)', 'storage_type_disc']

In [50]:
df_copy_final_test = df_copy.drop(columns=columnas_eliminar_df_copy_test)
df_copy_final_test.head(8)

,Company,TypeName,Inches,Ram,OpSys,Weight,Resolution,Pantalla,Pantalla_disc,Brand,Family,Model,Clock Speed (GHz),Gpu Brand,Gpu Model,Gpu Type,Storage Type,Componentes_principales
id,,,,,,,,,,,,,,,,,,
181,HP,Ultrabook,13.3,16,Windows 10,1.30,3840x2160,4K Ultra HD,4,Intel,Core i7,7500U,2.7,Intel,HD Graphics 620,Unknown,SSD,3.401197
708,Acer,Notebook,15.6,8,Windows 10,2.20,1366x768,Full HD,1,AMD,A12-Series,9720P,2.7,AMD,Radeon RX 540,Dedicated,SSD,2.944439
862,Acer,Notebook,15.6,4,Linux,2.40,1366x768,Full HD,1,Intel,Core i3,6006U,2.0,Intel,HD Graphics 520,Unknown,HDD,2.079442
1064,HP,Notebook,14.0,8,Windows 7,1.43,1920x1080,Full HD,1,Intel,Core i5,6200U,2.3,Intel,HD Graphics 520,Unknown,SSD,2.995732
702,HP,Notebook,13.3,8,Windows 10,1.34,1920x1080,Full HD,1,Intel,Core i5,7200U,2.5,Intel,HD Graphics 620,Unknown,SSD,2.890372
585,Dell,2 in 1 Convertible,15.6,16,Windows 10,2.00,1920x1080,Full HD,1,Intel,Core i7,8550U,1.8,Intel,UHD Graphics 620,Unknown,SSD,3.332205
195,Razer,Gaming,14.0,16,Windows 10,1.95,1920x1080,Full HD,1,Intel,Core i7,7700HQ,2.8,Nvidia,GeForce GTX 1060,Dedicated,SSD,3.637586
463,Asus,Notebook,15.6,4,Linux,2.10,1920x1080,Full HD,1,Intel,Core i5,7200U,2.5,Nvidia,GeForce 920,Dedicated,HDD,2.564949


## Exportación del df a un nuevo archivo

In [51]:
df_copy_final_test.to_csv('./data/df_test_precios_ordenadores.csv')
df_disc.to_csv('./data/df_test_disc_precios_ordenadores.csv')